<a href="https://colab.research.google.com/github/itsdayo/Pytorch-CNN/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Convert MNIST Image Files into a Tensor 4-Dimension (# of images, Height, Width, Color Channels)
transform = transforms.ToTensor()

In [ ]:
# Train Data

train_data = datasets.MNIST(root='/cnn_data', train=True, download =True, transform=transform)


100%|██████████| 9912422/9912422 [00:00<00:00, 88836800.02it/s]


Extracting /cnn_data/MNIST/raw/train-images-idx3-ubyte.gz to /cnn_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 54762971.89it/s]


Extracting /cnn_data/MNIST/raw/train-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 25437761.46it/s]


Extracting /cnn_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /cnn_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4934091.89it/s]


Extracting /cnn_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



In [ ]:
# Test Data
test_data = datasets.MNIST(root='/cnn_data', train=False, download =True, transform=transform)

In [ ]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [ ]:
# Creare a small batch size for images, lets say 10
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [ ]:
# Define Our Cnn Model
# Describe convolutional layer and what it's doing (2 convolutional layers)

conv1 = nn.Conv2d(1,6,3,1)
conv2 = nn.Conv2d(6,16,3,1)

In [ ]:
# Grab 1 MNIST record/image
for i ,(X_train, y_train) in enumerate(train_data):
  break

In [ ]:
X_train

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

In [ ]:
x = X_train.view(1,1,28,28)

In [ ]:
# Lets perform our first convolution
x = F.relu(conv1(x)) # Rectify Linear unit for activation function

In [ ]:
x.shape

torch.Size([1, 6, 26, 26])

In [ ]:
# Pass through the pooling layer
x = F.max_pool2d(x,2,2) #kernal (filter) of 2 stride of 2

In [ ]:
x.shape # 26/2 = 13

torch.Size([1, 6, 13, 13])

In [ ]:
# Do our secord convolutional layer

x = F.relu(conv2(x))

In [ ]:
x.shape # Didn't set padding lost 2 pixels in the outside

torch.Size([1, 16, 11, 11])

In [ ]:
x = F.max_pool2d(x,2,2)

In [ ]:
x.shape

torch.Size([1, 16, 5, 5])

In [ ]:
# Model Class
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,6,3,1)
    self.conv2 = nn.Conv2d(6,16,3,1)
    # Full connected layer
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  def forward(self, X):
    X = F.relu(self.conv1(X))
    X= F.max_pool2d(X,2,2)
    # Second path
    X = F.relu(self.conv2(X))
    X= F.max_pool2d(X,2,2)

    # Re-view to flatten it out
    X = X.view(-1,16*5*5) # negative one so we can vary the batch size

    # Fully connected layers
    X = F.relu(self.fc1(X))
    X =F.relu(self.fc2(X))
    X = self.fc3(X)
    return F.log_softmax(X,dim=1)


In [ ]:
# Create an instance of our model
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
# Loss function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Epoch: 0 Batch: 600 Loss: <built-in method item of Tensor object at 0x785b922318a0>
Epoch: 0 Batch: 1200 Loss: <built-in method item of Tensor object at 0x785b91fcbbf0>
Epoch: 0 Batch: 1800 Loss: <built-in method item of Tensor object at 0x785b91fcbbf0>
Epoch: 0 Batch: 2400 Loss: <built-in method item of Tensor object at 0x785b91fc8ef0>
Epoch: 0 Batch: 3000 Loss: <built-in method item of Tensor object at 0x785b91fc9ee0>
Epoch: 0 Batch: 3600 Loss: <built-in method item of Tensor object at 0x785b91fcb1a0>
Epoch: 0 Batch: 4200 Loss: <built-in method item of Tensor object at 0x785b91fc8c20>
Epoch: 0 Batch: 4800 Loss: <built-in method item of Tensor object at 0x785b91fca750>
Epoch: 0 Batch: 5400 Loss: <built-in method item of Tensor object at 0x785b91fcb2e0>
Epoch: 0 Batch: 6000 Loss: <built-in method item of Tensor object at 0x785b91fcb2e0>
Epoch: 1 Batch: 600 Loss: <built-in method item of Tensor object at 0x785b91fc9210>
Epoch: 1 Batch: 1200 Loss: <built-in method item of Tensor object a